<a href="https://colab.research.google.com/github/shiyuhu1933/EC-523-final-project/blob/main/hdf5_to_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import torch
import os
import h5py
import numpy as np

drive.mount('/content/gdrive')

DATASET_GOOGLE_DRIVE_PATH = '/content/gdrive/MyDrive/CS523/Project/dataset'


def get_configuration(batch_size):
    config = {}
    config['batch_size'] = batch_size
    config['num_workers'] = 1
    return config

def get_file_name(files_list):
  with open(files_list) as f:
    return [line.rstrip()[:] for line in f]

class FaceMaskData(torch.utils.data.Dataset):

  def __init__(self, mode, config, transform=None):

    self.data_dir = DATASET_GOOGLE_DRIVE_PATH

    if mode == 'train':
      self.files = get_file_name(os.path.join(self.data_dir, 'train_files.txt'))
    else:
      self.files = get_file_name(os.path.join(self.data_dir, 'test_files.txt'))

    image = []
    label = []

    for dataset in self.files:
      path = os.path.join(self.data_dir, dataset)
      self.file = h5py.File(path, 'r')
      self.total_num_imgs, self.H, self.W, self.C = self.file['image'].shape
      image.append(self.file['image'][:])
      label.append(self.file['labels'][:])

    self.image = np.vstack(image)
    self.label = np.vstack(label)

    self.num_images = len(self.image) 

  def __getitem__(self, index):
    """Return one image and its corresponding attribute label."""
    image = self.image[index]
    label = self.label[index]
    if self.transform:
        image = self.transform(image)
    return image, torch.FloatTensor(label)

  def __len__(self):
    return self.num_images
      



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
config = get_configuration(16)
dataset = FaceMaskData('train', config)
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                              batch_size=config['batch_size'],
                              shuffle=True, 
                              num_workers=config['num_workers'])

for i, data in enumerate(data_loader, 0):
  image, label = data
  image, label = image.cuda(), label.cuda()
  print(image)

AttributeError: ignored

In [ ]:
def get_loader(config, mode='train'):

  batch_size = config['batch_size']
  num_workers = config['num_workers']

  dataset = FaceMaskData(mode, config)
  data_loader = torch.utils.data.DataLoader(dataset=dataset,
                              batch_size=batch_size,
                              shuffle=True, 
                              num_workers=num_workers)
  return data_loader

In [ ]:
from torchvision.transforms import ToPILImage
to_img = ToPILImage()

# display tensor
print(input[0].shape)
to_img( input[0] )